<a href="https://colab.research.google.com/github/scope-lab-vu/transactive-blockchain/blob/master/TCPS_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To make this notebook work in Colab, the following must be done:

  1. The Cplex binary installer needs to be uploaded to your Google Drive in a directory named 'ColabLibraries'
  2. Prosumer offer data should be uploaded into a directory named 'profiles'

Mount Google Drive to access installation files

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Install cplex from Google Drive. installer.properties included in directory for silent install

In [2]:
! chmod u+x /content/gdrive/My\ Drive/ColabLibraries/cplex_studio129.linux-x86-64.bin

!/content/gdrive/My\ Drive/ColabLibraries/cplex_studio129.linux-x86-64.bin -f "installer.properties"


Preparing to install
Extracting the JRE from the installer archive...
Unpacking the JRE...
Extracting the installation resources from the installer archive...
Configuring the installer for this system's environment...

Launching installer...



Install cplex python binaries

In [0]:
%%capture
! cd /opt/ibm/ILOG/CPLEX_Studio129/python/; python3 setup.py install;

Clone transax repo

In [4]:
from getpass import getpass
import os

user = getpass('GitHub User')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password

!git clone https://$GITHUB_AUTH@github.com/scope-lab-vu/transactive-blockchain.git


GitHub User··········
GitHub password··········
Cloning into 'transactive-blockchain'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 4771 (delta 16), reused 4 (delta 1), pack-reused 4740
Receiving objects: 100% (4771/4771), 139.49 MiB | 34.45 MiB/s, done.
Resolving deltas: 100% (2184/2184), done.
Checking out files: 100% (2097/2097), done.


Install transax library

In [0]:
%%capture
! cd /content/transactive-blockchain/transax; sudo pip3 install .

Set up the microgrid and the solver

In [0]:
import cplex
from transax.Microgrid import Microgrid
from transax.MatchingSolver import Offer, MatchingSolver
microgrid = Microgrid(interval_length=1.0, C_ext=25000, C_int=250000, feeders=[0], prosumer_feeder={
    101: 0,
    102: 0,
    103: 0,
    104: 0,
    105: 0,
    201: 0,
    202: 0,
    203: 0,
    204: 0,
    207: 0,
    301: 0,
    302: 0,
    303: 0,
    304: 0,
    305: 0,
  })
solver = MatchingSolver(microgrid)

Build a dictionary of all the prosumer offers

In [0]:
import pandas as pd
import numpy as np
prosumer_df = {}
directory = os.fsencode('/content/gdrive/My Drive/profiles')
for file in os.listdir(directory):
  filename = os.fsdecode(file)
  prosumer_id = int(filename.split('_')[1].split('.')[0])
  prosumer_df[prosumer_id] = pd.read_csv('/content/gdrive/My Drive/profiles/' + filename)


Create lists for buying and selling offers, also determine total energy on the grid. Consumption is represented with positive values, production is represented with negative values.

In [0]:
buying_offers = []
selling_offers = []
total_energy = 0
for key, prosumer in prosumer_df.items():
  for row in prosumer.iterrows():
      startTime = int(row[1]['startTime'])
      endTime = int(row[1]['endTime'])
      energy = float(row[1]['energy'])
      total_energy = total_energy + energy
      offer = Offer(key, key, startTime, endTime, np.abs(energy), 1)
      if energy < 0:
        buying_offers.append(offer)
      elif energy > 0:
        selling_offers.append(offer)

Determine total energy traded after all offers have been matched

In [9]:
(trades, objective) = solver.solve(buying_offers=buying_offers, selling_offers=selling_offers)
print("Success: {} energy traded".format(objective))

Success: 765.8317905248687 energy traded


Case 1: All prosumers produce/consume exactly what they offered. Find total demand on DSO.



In [10]:
unmet_energy = np.abs(total_energy)-objective
print('Prosumers produce as anticipated. Load on DSO: {}'.format(unmet_energy))

Prosumers produce as anticipated. Load on DSO: 157.58606322981495


Case 2: Prosumers' offers are all incorrect by a normal distribution.

In [11]:
total_energy = 0
for prosumer in prosumer_df.values():
  for row in prosumer.iterrows():
      energy = float(row[1]['energy']) + np.random.normal(0,1)
      total_energy = total_energy + energy

unmet_energy = np.abs(total_energy)-objective
print('Each offer is incorrect by a normal distribution. Load on DSO: {}'.format(unmet_energy))

Each offer is incorrect by a normal distribution. Load on DSO: 194.2489863146177


Case 3: Producers consistently overestimate their production ability. (i.e. a cloudy day that was supposed to be clear)

In [12]:
total_energy = 0
for prosumer in prosumer_df.values():
  for row in prosumer.iterrows():
      energy = float(row[1]['energy'])
      if energy > 0:
        energy = energy - np.random.exponential(2)
      total_energy = total_energy + energy

unmet_energy = np.abs(total_energy)-objective
print('Producers consistently overestimate production. Load on DSO: {}'.format(unmet_energy))

Producers consistently overestimate production. Load on DSO: 1281.0863287820382


Plot of loss of trading volume vs. offer error


In [17]:
sigmas = [1, 2, 3, 4, 5]
total_energy = np.zeros(len(sigmas))
for idx, sigma in enumerate(sigmas):
  for prosumer in prosumer_df.values():
    for row in prosumer.iterrows():
        energy = float(row[1]['energy']) + np.random.normal(0,sigma)
        total_energy[idx] = total_energy[idx] + energy

unmet_energy = np.abs(total_energy)-objective
print('Each offer is incorrect by a normal distribution. Load on DSO: {}'.format(unmet_energy))

Each offer is incorrect by a normal distribution. Load on DSO: [135.61039952 196.76930199 148.94222563 -31.41597801 196.08214677]


In [14]:
import matplotlib.pyplot as plt



array([[-903.38112553, -903.38112553, -903.38112553, -903.38112553,
        -903.38112553]])